In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import mysql.connector
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from user_agent import generate_user_agent, generate_navigator
from selenium.common.exceptions import UnexpectedAlertPresentException
import requests
import datetime

In [2]:
def movie_crawling(webdriver_path, service, driver, id):
    webdriver_path = webdriver_path
    service = service
    driver = driver
    try:
        url = f'http://cine21.com/movie/info/?movie_id={id}'
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        sub_info = soup.find_all('p', class_='sub_info')
        sub_info_list = [i.text.strip().replace('\n', ' ').replace('\t', ' ') for i in sub_info]
        ko_name = soup.find('p', class_='tit').text
        en_name = soup.find('p', class_='tit_eng').text.strip().split('\n')[-1]
        story = soup.find('div', class_='story').text.strip().replace('\t', '').replace('\n', ' ').replace('  ', ' ')
        if story == '':
            story = None
        year = sub_info_list[0].split()[0]
        country = sub_info_list[0].split()[1]
        poster_link = soup.find('div', class_='mov_poster').find('img')['src']
        if poster_link == 'https://image.cine21.com/resize//noimg[X230,330].jpg':
            poster_link = None

        try:
            if sub_info_list[2][:3] == '개봉일' and len(sub_info_list[2][6:16]) == 10:
                pd.to_datetime(sub_info_list[2][6:16])
                release_date = sub_info_list[2][6:16]
            else:
                release_date = None
        except:
            release_date = None
    
        if sub_info_list[2][17:21] == '누적관객':
            audience = int(sub_info_list[2][24:].replace(',', '').replace('명', '').replace(',', ''))
        else:
            audience = None
        
        if sub_info_list[1].split('상영시간 : ')[0] != '':
            genre = sub_info_list[1].split('상영시간 : ')[0][:-1]
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])
        else:
            genre = None
            running_time = int(sub_info_list[1].split('상영시간 : ')[1][:-1])

        if sub_info_list[3][0:2] == '감독':
            director = sub_info_list[3][18:]
        else:
            director = None

        for i in range(1, 6):
            if soup.find('span', class_=f'grade00{i}') != None:
                grade = soup.find('span', class_=f'grade00{i}').text
                break
            elif i == 5:
                grade = None

        return [id, ko_name, en_name, year, country, grade, genre, running_time, release_date, audience, director, story, poster_link]
    
    except UnexpectedAlertPresentException as e1:
        return e1.alert_text
    
    except IndexError as e2:
        return '영화 정보 불충분'
    
    except AttributeError as e3:
        return '일시적으로 서비스 장애가 발생하였습니다.'

In [3]:
create_table_query = """
CREATE TABLE IF NOT EXISTS movies (
    id INT PRIMARY KEY,
    ko_name VARCHAR(255),
    en_name VARCHAR(255),
    country VARCHAR(255),
    grade VARCHAR(255),
    genre VARCHAR(255),
    runtime INT,
    release_date DATE,
    audience INT,
    director VARCHAR(255),
    story VARCHAR(255),
    poster_link VARCHAR(255))"""

In [4]:
# # My SQL 데이터베이스 연결
# conn = mysql.connector.connect(
#     host = 'localhost',
#     user = 'root',
#     password = 'signcity114',
#     database = 'MyDB'
# )

# cursor = conn.cursor()

# cursor.execute(create_table_query)
# conn.commit()
# print("테이블이 성공적으로 생성되었습니다")

# conn.close()

In [35]:
# My SQL 데이터베이스 연결
conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'signcity114',
    database = 'MyDB'
)

# 연결 확인
if conn.is_connected():
    print("MySQL에 성공적으로 연결되었습니다")
else:
    print("MySQL 연결 실패")

cursor = conn.cursor()

webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)

insert_query = """
INSERT INTO movie
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

for i in range(28001, 30001):
    data = movie_crawling(webdriver_path, service, driver, i)
    if data == '존재하지 않는 영화입니다.':
        print(data)
        continue
    elif data == '성인용 컨텐츠입니다. 로그인 해 주세요.':
        print(data)
        continue
    elif data == '영화 정보 불충분':
        print(data)
        continue
    elif data == '일시적으로 서비스 장애가 발생하였습니다.':
        print(data)
        continue
    cursor.execute(insert_query, data)
    conn.commit()
    print(data)

MySQL에 성공적으로 연결되었습니다
[27675, '전시의 겨울', 'Winter in Wartime', '2008', '벨기에,네덜란드', None, '전쟁', 104, None, None, '마틴 쿨호벤', '2차대전이 거의 끝나가던 무렵, 13세의 미하엘은 부상당한 영국 군인을 도와준 것을 계기로 레지스탕스에 가담하게 된다. 전쟁이 끝나고 점차 성장해 가면서 미하엘은 상상 속의 모험과 전쟁의 추악한 실태간에 어마어마한 차이가 있음을 깨닫게 된다.', None]
[27676, '모래바람, 바위여인', 'Winds of Sand, Women of Rock', '2009', '프랑스,오스트리아,벨기에', None, '다큐멘터리', 90, None, None, '나탈리 보르제르', '아프리카 사하라 이남의 니제르. 매년 비가 내리고 나면 여성들은 낙타를 끌고 사막을 가로지르는 험난한 길을 떠난다. 문명화된 세계에 대한 동경, 여성으로서 자아성취를 이루고자 하는 소망 등이 세 명의 여성을 중심으로 그들의 대화를 통해 펼쳐진다. 하늘, 땅의 경계가 불분명한 무채색 사막을 꿋꿋이 뚫고 나가는 바위 같은 여성들의 현재와 미래를 그린 다큐멘터리. 광활한 풍경 속 다채로운 옷차림의 인물들, 사막의 일몰, 둥근 보름달을 찍어내는 감각적인 영상미는 눈을 떼기 힘들 정도다. 문명화된 세계의 한쪽 끝에서, 자연이 주는 불편에 순응하며 겸허한 삶을 영위하는 이들의 존재를 새삼 상기하게 되는 동시에, 전통의 고수와 자아실현 사이에 놓인 그곳 여성들의 고민에 주목하게 된다.', None]
[27677, '백색 공간', 'The White Space', '2009', '이탈리아', None, '드라마', 98, None, None, '프란체스카 코멘치니', '마리아는 이혼 후 나름대로 꿋꿋이 싱글의 삶을 살아가는 나폴리의 야학 교사다. 영화를 보러 갔다가 새 애인이 생기면서 달콤한 순간들을 보내지만, 예기치 못한 임신을 하게 되자 고민 끝에 헤어진다. 아이를 조산하게 된 그녀는 인큐베

WebDriverException: Message: unknown error: net::ERR_CONNECTION_TIMED_OUT
  (Session info: chrome=129.0.6668.101)
Stacktrace:
0   chromedriver                        0x0000000104dac200 cxxbridge1$str$ptr + 1907280
1   chromedriver                        0x0000000104da46e8 cxxbridge1$str$ptr + 1875768
2   chromedriver                        0x00000001049b8260 cxxbridge1$string$len + 89488
3   chromedriver                        0x00000001049b17bc cxxbridge1$string$len + 62188
4   chromedriver                        0x00000001049a3654 cxxbridge1$string$len + 4484
5   chromedriver                        0x00000001049a508c cxxbridge1$string$len + 11196
6   chromedriver                        0x00000001049a3a00 cxxbridge1$string$len + 5424
7   chromedriver                        0x00000001049a327c cxxbridge1$string$len + 3500
8   chromedriver                        0x00000001049a3208 cxxbridge1$string$len + 3384
9   chromedriver                        0x00000001049a159c core::str::slice_err

In [54]:
webdriver_path = '/Users/anhyojun/WebDriver/chromedriver'
service = Service(webdriver_path)
driver = webdriver.Chrome(service=service)
movie_crawling(webdriver_path, service, driver, 415)

[415,
 '언지프',
 'Unzipped',
 '미국',
 '15세이상관람가',
 '다큐멘터리',
 73,
 None,
 7903,
 '더글러스 키브',
 '흥미진진하고 즐거움을 주는 매우 사적인 패션 디자이너 마즈라히는 Nanlook of the North, Samantha Stevens 등 다양한 문화계의 우상들에게 영감을 얻으면서 좌절과 기쁨과 공포로 가득한 모델들이 걸어다니는 진짜 패션쇼를 만들어내기 위해 분주하다. 그는 중요한 잡지 편집자를 위해 자신의 디자인을 재검토하고 루브르에서 기록으로 남아 있는 패션 콜렉션을 보기 위해 파리로 여행하기도 한다.  쇼가 가까워지자 그는 디자인에 생명을 불어넣고, 케이트 못, 린다 에반젤리스타, 신디 크로포드 같은 세계 톱모델들과 계약을 체결하는 과정을 거치면서 유명인사들과 비평가들로 들어찬 쇼장에서 대담하고 복잡미묘하게 신경을 곤두서게 하는 쇼를 진행한다.',
 'https://image.cine21.com/resize/cine21/poster/2005/0615/M0010197_[X230,330].jpg']